<a href="https://colab.research.google.com/github/feliperoque/Data_Science_Analysis/blob/master/An%C3%A1lise_de_Dados_de_Homic%C3%ADdios_por_Estupro_no_Brasil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Análise dos Dados de Homicídios por Estupro no Brasil**



---



* Inicialmente dentro dessa análise, vamos: 

    1. Realizar o pré-processamento dos dados referente ao Brasil;
    2. Realizar o pré-processamento dos dados referente a Região;
    3. Realizar o pré-processamento dos dados referente a unidade federativa;
    4. Analisar os dados referentes ao homicídios por estupro no ***Brasil***;
    5. Analisar dos dados referentes ao homicídios de estupro em cada ***região do Brasil***;
    6. Analisar os dados referentes ao homicídios de estupro por ***Unidade Federativa (UF)***;


**OBS**: Os dados são proveninentes do IPEA (instituto de pesquisa econômica aplicada) via requisição de sua própria api disponibilizada em seu site. Para mais informações visite https://www.ipea.gov.br/atlasviolencia/api.



---



# 1) **Pré-Processamento dos Dados**:



---



In [1]:
import urllib3
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px

# DADOS DO ESTUPRO:
#
# Estupro no Brasil --> https://www.ipea.gov.br/atlasviolencia/api/v1/valores-series/89/1 
# Estupro por Região --> https://www.ipea.gov.br/atlasviolencia/api/v1/valores-series/89/2
# Estupro por UF --> https://www.ipea.gov.br/atlasviolencia/api/v1/valores-series/89/3
# Estupro por Município --> Indisponível

def DATA_ESTUPRO_BRASIL(URL):
    http = urllib3.PoolManager()
    data_estupro = http.request("GET", URL)

    if data_estupro.status == 200:
        data = data_estupro.data.decode('utf-8')
        data_json = json.loads(data)
        df_1 = pd.DataFrame(data_json)

    else:
        data_estupro.raise_for_status()

    return df_1

def DATA_ESTUPRO_REGIAO(URL):
    http = urllib3.PoolManager()
    data_estupro = http.request("GET", URL)
    
    if data_estupro.status == 200:
        data = data_estupro.data.decode('utf-8')
        data_json = json.loads(data)
        df_2 = pd.DataFrame(data_json)

    else:
        data_estupro.raise_for_status()

    return df_2

def DATA_ESTUPRO_UF(URL):
    http = urllib3.PoolManager()
    data_estupro = http.request("GET", URL)
    
    if data_estupro.status == 200:
        data = data_estupro.data.decode('utf-8')
        data_json = json.loads(data)
        df_3 = pd.DataFrame(data_json)

    else:
        data_estupro.raise_for_status()

    return df_3


API_ESTUPRO_BRASIL = 'https://www.ipea.gov.br/atlasviolencia/api/v1/valores-series/89/1'
API_ESTUPRO_REGIAO = 'https://www.ipea.gov.br/atlasviolencia/api/v1/valores-series/89/2'
API_ESTUPRO_UF = 'https://www.ipea.gov.br/atlasviolencia/api/v1/valores-series/89/3'

data_brasil = DATA_ESTUPRO_BRASIL(API_ESTUPRO_BRASIL)
data_regiao = DATA_ESTUPRO_REGIAO(API_ESTUPRO_REGIAO)
data_uf = DATA_ESTUPRO_UF(API_ESTUPRO_UF)


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


* Temos agora três dataframes:

  1. **data_brasil** referente a série temporal nos periodos de 2011 a 2016;

  2. **data_regiao** referente a série temporal nos periodos de 2011 a 2016 por região do país;

  3. **data_uf** referente a série temporal nos peridos de 2011 a 2016 por unidade federativa;

* Vamos iniciar nossa análise no que se refere aos dados contidos em ***data_brasil*** exibindo primeiro a estrutura do seu dataframe.

### 1.1) **Pré-processamento do Dataframe `data_brasil`**:

* Realizando uma pequena observação preliminar do cabeçalho do dataframe. Percebemos que há 4 colunas:

  * `cod`: contém o código (desnecessário para nossa análise);

  * `sigla`: contém as sigla do país que estamos trabalho (no caso o Brasil);

  * `valor`: contém os valores referentes a quantidade de  vítimas;

  * `periodo`: contém as datas anuais de 2011 à 2016.



In [3]:
data_brasil.head() 

,cod,sigla,valor,periodo
0,1076,BRA,12087,2011-01-15
1,1076,BRA,16104,2012-01-15
2,1076,BRA,19129,2013-01-15
3,1076,BRA,20026,2014-01-15
4,1076,BRA,20597,2015-01-15


* Analisando a quantidade de dados que estão contidos nas colunas `sigla`, `valor` e `periodo`, verifica-se que há 6 valores para cada coluna:


In [4]:
data_brasil.count() 

cod        6
sigla      6
valor      6
periodo    6
dtype: int64

* Podemos verificar qual o tipo de dado cada coluna recebe. Após a verificão do tipo de dado que cada coluna recebe, percebemos que há valores do tipo `object` na coluna em todas as colunas.

    * **cod**: `object`
    * **sigla**: `object`
    * **valor**: `object`
    * **periodo**: `object`

* Iremos converter o tipo de dado da coluna `valor`  de `object` para `int64` e o tipo de dado da coluna `periodo` para o formato `datetime`. Pois só assim conseguiremos gerar os gráficos posteriormente.

In [5]:
data_brasil.dtypes

cod        object
sigla      object
valor      object
periodo    object
dtype: object

In [2]:
data_brasil['valor'] = data_brasil['valor'].astype('int')
data_brasil['periodo'] = pd.to_datetime(data_brasil['periodo']).dt.date

* Também iremos renomear as seguintes colunas em nosso dataframe `data_brasil`:

  1. `periodo` para `Ano`
  2. `valor` para `Quantidade de Vítimas`

In [3]:
data_brasil = data_brasil.rename(columns={'valor':'Quantidade de Vítimas', 'periodo': 'Ano'})

* Podemos remover a coluna `cod` (pois não haverá serventia em nossa análise):

In [4]:
data_brasil = data_brasil.drop(['cod'],axis=1)



---



## 1.2) Pré-processamento do Dataframe `data_regiao`:

* Observando o cabeçalho do dataframe `data_regiao`, podemos verificar que temos 4 colunas:

  * `cod`: contém o código (desnecessário para nossa análise);
  * `sigla`: contém as sigla dos estados brasileiros;
  * `valor`: contém os valores referentes a quantidade de vítimas;
  * `periodo`: contém as datas anuais de 2011 à 2016.


In [9]:
data_regiao.head() 

,cod,sigla,valor,periodo
0,4,S,2216,2011-01-15
1,2,NE,1897,2011-01-15
2,1,N,2599,2011-01-15
3,5,CO,1107,2011-01-15
4,3,SE,4268,2011-01-15


* Verificando a quantidade de dados que estão contidos nas colunas `sigla`, `valor` e `periodo`. Vemos que nas 4 colunas temos 30 valores em cada.



In [11]:
data_regiao.count() 

cod        30
sigla      30
valor      30
periodo    30
dtype: int64

* Analisando os tipos de dados que cada coluna contém, percebemos que há valores do tipo `object` em todas as colunas. Com isso é necessário converter a coluna `valor` do tipo `object` para `int64` e a coluna `periodo` de `object` para `datetime`, caso contrário, não conseguiremos gerar gráficos com esses tipos de dados.

In [12]:
data_regiao.dtypes

cod        object
sigla      object
valor      object
periodo    object
dtype: object

In [5]:
data_regiao['valor'] = data_regiao['valor'].astype('int')
data_regiao['periodo'] = pd.to_datetime(data_regiao['periodo']).dt.date

* Vamos renomear o nome das colunas `valor` para `Quantidade de Vítimas `e `periodo` para `Ano`. E remover a coluna `cod`, pois não haverá serventia em nossa análise:

In [6]:
data_regiao = data_regiao.rename(columns={'valor':'Quantidade de Vítimas', 'periodo': 'Ano'})
data_regiao = data_regiao.drop(['cod'], axis=1)



---



## 1.3) Pré-processamento do Dataframe `data_uf`:



---



* O dataframe referente a unidade federativa, contém o mesmo layout ods anteriores, a única diferênça é que a coluna `sigla`, contém as siglas dos estados.

In [7]:
data_uf.head(5) # Retorna o cabeçalho do dataframe

,cod,sigla,valor,periodo
0,13,AM,855,2011-01-15
1,28,SE,201,2011-01-15
2,12,AC,280,2011-01-15
3,23,CE,68,2011-01-15
4,14,RR,112,2011-01-15


* Verificando a quantidade de dados que estão contidos nas colunas` sigla`, `valor` e `periodo`, temos 162 valores para todos.

In [18]:
data_uf.count()

cod        162
sigla      162
valor      162
periodo    162
dtype: int64

* Verificando também os tipos de dados, vemos que temos em todas as colunas o do tipo `object` e precisaremos mudar esse tipo para `int64`, na coluna `valor`. Já para a coluna `periodo`, mudaremos para o tipo `datetime`.

In [19]:
data_uf.dtypes

cod        object
sigla      object
valor      object
periodo    object
dtype: object

In [9]:
data_uf['valor'] = data_uf['valor'].astype('int')
data_uf['periodo'] = pd.to_datetime(data_uf['periodo']).dt.date

* Renomearemos o nome das colunas `valor` para `Quantidade de Vítimas ` e `periodo` para `Ano`:

In [10]:
data_uf = data_uf.rename(columns={'valor':'Quantidade de Vítimas', 'periodo': 'Ano'})

* Também iremos remover a coluna `cod` (pois não haverá serventia em nossa análise):

In [11]:
data_uf = data_uf.drop(['cod'], axis=1)



---



# 2) **Análise Gráfica de Homicídios por Estupro**:

### 2.1) **Análise de Homicídios por Estupro no País**:






---



* A partir do gráfico de barra entre os dados da coluna `Ano x Quantidade de Vítimas`, pudermos visualizar a quantidade de vítimas de homicidio por estupro no período de 2011 a 2016.

 



In [ ]:
fig = px.bar(data_brasil,
             x='Ano', 
             y='Quantidade de Vítimas',
             title='Ano x Quantidade de Vítimas')
fig.show()

* A partir do gráfico podemos concluir que:

  1. Ouve um crescente aumento de casos de homicídio por estupro no brasil ao longo dos anos. 

* Com a função `describe()` podemos visualizar algumas informações estatísticas como:

  2.  A `média` de casos de 2011-2016 foi de 18.476 mil vítimas;

  3. O `valor mínimo` foi em 2011 com cerca de 12.087 mil vítimas;

  4. O `valor máximo` foi em 2016 com cerca de 22.918 mil vítimas.

In [ ]:
data_brasil.describe()

,valor
count,6.000000
mean,18476.833333
std,3834.178265
min,12087.000000
25%,16860.250000
50%,19577.500000
75%,20454.250000
max,22918.000000




---



### 2.2) **Análise de Homicídios por Estupro por Região**:



---



* Iniciando nossa análise por um gráfico de pizza clássico, podemos verificar qual a região possui a maior parcela de homicídios por estupro.

In [ ]:
data_regiao2 = data_regiao.groupby(['sigla']).sum()

In [ ]:
data_regiao2

,cod,valor,periodo
sigla,,,
CO,555555,110715392004189619412250,2011-01-152012-01-152013-01-152014-01-152015-0...
N,111111,259933193898402238374072,2011-01-152012-01-152013-01-152014-01-152015-0...
NE,222222,189724973422369239023998,2011-01-152012-01-152013-01-152014-01-152015-0...
S,444444,221630313556384639884263,2011-01-152012-01-152013-01-152014-01-152015-0...
SE,333333,426857186249657069298335,2011-01-152012-01-152013-01-152014-01-152015-0...


In [ ]:
fig = px.pie(data_regiao2,
             values='Quantidade de Vítimas',
             names=data_regiao2.index,
             title='Homicídio por Estupro por Região: jan/2011 à jan/2016', 
             color_discrete_sequence=px.colors.sequential.RdBu)

fig.update_traces(textposition='inside',
                  textinfo='percent+label')

fig.show()

* Com a visualização do gráfico de pizza acima, podemos extrair as seguintes informações:

  1. A região que possui um maior índice de homicídios por estupro é a região sudeste com 34,3% (dentro do período analisado).
  
  2. Em segundo vem a região Norte com 19,6% (dentro do período analisado);

  3. Em terceiro a região Sul com 18,9% (dentro do período analisado);

  4. Em quarto a região Nordeste com 17,5% e em quinto a região Centro-Oeste com a menor taxa de 9,69% (dentro do período analisado).

* Vale salientar que houve o somatório dos homicídios por estupro durante os períodos de jan/2011 à jan/2016 para que se fosse possível gerar o gráfico.

* Outro gráfico que pode trazer informações em relação a quantidade de homicídios em um determinado ano por região é por meio do `treemap chart`.

In [ ]:
data_regiao3 = data_regiao.groupby(['sigla',
                                    'Ano']).sum().reset_index()
                                    
fig1 = px.treemap(
    data_regiao3,
    path=['sigla',
          'Ano',
          'Quantidade de Vítimas'],
    color_discrete_sequence = px.colors.sequential.RdBu
    )

fig1.show()

* Com o `treemap chart` gerado, podemos extrair que:

  * **Região Centro-Oeste**:
    * Há um oscilação de homicídios por estupro nos períodos de jan/2011 à jan/2016 (5 anos).

    * Se pegarmos o mesmo período citado (jan/2011 à jan/2016) houve um aumento de aproximadamente 1.143 vítimas.

  * **Região Norte**:
    * Há um oscilação de homicídios por estupro nos períodos de jan/2011 à jan/2016 (5 anos).

    * Se pegarmos o mesmo período citado (jan/2011 à jan/2016) houve um aumento de aproximadamente 1.473 vítimas.

  * **Região Sul**:
    * Há um oscilação de homicídios por estupro nos períodos de jan/2011 à jan/2016 (5 anos).

    * Se pegarmos o mesmo período citado (jan/2011 à jan/2016) houve um aumento de aproximadamente 2.047 vítimas.

  * **Região Nordeste**:
    * Há um oscilação de homicídios por estupro nos períodos de jan/2011 à jan/2016 (5 anos).

    *  Se pegarmos o mesmo período citado (jan/2011 à jan/2016) houve um aumento de aproximadamente 2.101 vítimas.

  * **Região Sudeste**:
    * Há um oscilação de homicídios por estupro nos períodos de jan/2011 à jan/2016 (5 anos).

    * Se pegarmos o mesmo período citado, houve um aumento de aproximadamente 4.067 vítimas.



* Com isso, a partir do `treemap char`t concluirmos que a região que teve o maior aumento no número de casos de homicídos por estupro, foi a região Suldeste.



---



### 2.3) **Análise de Homicídios por Estupro por UF**:



---



* Um gráfico que pode elucidar mais uma vez a situação de homicídios por estupro é por meio do `treemap chart`:

In [ ]:
data_uf.head(5)

,sigla,Quantidade de Vítimas,Ano
0,AM,855,2011-01-15
1,SE,201,2011-01-15
2,AC,280,2011-01-15
3,CE,68,2011-01-15
4,RR,112,2011-01-15


In [ ]:
data_uf1 = data_uf.groupby(['sigla',
                            'Ano']
                           ).sum().reset_index()

fig1 = px.treemap(
    data_uf1,
    path=['sigla',
          'Ano',
          'Quantidade de Vítimas'],
          color_discrete_sequence = px.colors.qualitative.D3)

fig1.update_layout(margin = dict(t=50,
                                 l=25,
                                 r=25,
                                 b=25))

fig1.show()

* Com o `treemap chart` gerado, podemos realizar a extração de infomações:

    * Acre:
        *  De 2011 à 2016 ouve um aumento de 168 casos de homicidio por estupro;
        
    * Amapá:
        * De 2011 à 2016 ouve um aumento de 82 casos de homicídio por estupro;

    * Distrito Federal:
        * De 2011 à 2016 ouve um aumento de 409 casos de homicídio por estupro;

    * Mato Grosso:
        * De 2011 à 2016 ouve um aumento de 36 casos de homicídio por estupro;

    * Pernambuco:
        * De 2011 à 2016 ouve um aumento de 439 casos de homicídio por estupro;

    * Rio de Janeiro:
        *  De 2011 à 2016 ouve um aumento de 914 casos de homicídio por estupro;

    * Roraima:
        *  De 2011 à 2016 ouve um aumento de 44 casos de homicídio por estupro;

    * Rio Grande do Sul:
        *  De 2011 à 2016 ouve um aumento de 325 casos de homicídio por estupro;

    * Alagoas:
        *  De 2011 à 2016 ouve um aumento de 127 casos de homicídio por estupro;

    * Espirito Santo:
        * De 2011 à 2016 ouve um aumento de 437 casos de homicídio por estupro;

    * Minas Gerais:
        * De 2011 à 2016 ouve um aumento de 1059 casos de homicídio por estupro;

    * Pará:
        * De 2011 à 2016 ouve um aumento de 354 casos de homicídio por estupro;

    * Piauí:
        * De 2011 à 2016 ouve um aumento de 544 casos de homicídio por estupro;

    * Rio Grande do Norte:
        * De 2011 à 2016 ouve um aumento de 55 casos de homicídio por estupro;

    * Santa Catarina:
        * De 2011 à 2016 ouve um aumento de 442 casos de homicídio por estupro;

    * São Paulo:
        * De 2011 à 2016 ouve um aumento de 1657 casos de homicídio por estupro;

    * Amazonas:
        * De 2011 à 2016 ouve um aumento de 227 casos de homicídio por estupro;

    * Ceará:
        * De 2011 à 2016 ouve um aumento de 366 casos de homicídio por estupro;

    * Goias:
        * De 2011 à 2016 ouve um aumento de 604 casos de homicídio por estupro;

    * Mato Grosso do Sul:
        * De 2011 à 2016 ouve um aumento de 91 casos de homicídio por estupro;

    * Paraiba:
        * De 2011 à 2016 ouve uma redução de -13 casos de homicídio por estupro;

    * Paraná:
        * De 2011 à 2016 ouve um aumento de 1280 casos de homicídio por estupro;

    * Rondônia:
        * De 2011 à 2016 ouve um aumento de 168 casos de homicídio por estupro;

    * Sergipe
        * De 2011 à 2016 ouve uma redução de -64 casos de homicídio por estupro;

    * Tocantins:
        * De 2011 à 2016 ouve um aumento de 435 casos de homicídio por estupro;

* houve uma redução deste tipo de crime nos estados da Paraiba e Sergipe mas no restante do país, só houveram aumento.

* Por meio do gráfico de barra na horizontal, podemos extrair as seguintes informações:

  * O estado do Brasil que possui maior caso de homicídio por estupro no período de 2011 à 2016 é o do estado de São Paulo, em segundo vem o estado de Mato Grosso e em terceiro o do Pará.



In [ ]:
data_uf2 = data_uf.groupby(['sigla']).sum().reset_index()

In [ ]:
fig4 = px.bar(data_uf2,
              x="Quantidade de Vítimas",
              y="sigla",
              orientation='h')

fig4.show()



---

